In [1]:
%load_ext autoreload
%autoreload 2
from llm_utils import limp_llm_plan, build_intent_dict, generate_json_with_gpt

In [2]:
## Test instructions:
"Pick up the book on the table with the red cover."
"Place the vase near the flowers on the table."
"Place the coke can beside the pringles on the counter."
"Bring the book and the magazine on the nightstand."
"Bring the mug from the table near the sink."

'Bring the mug from the table near the sink.'

In [3]:
## Test example
# Transcript: place the coke can beside the pringles on the counter

# interpretation_1
# "Place the coke can (which is currently beside the pringles) on some counter."
intent_labels_1 = {"place": 0, "the": 0, "coke": 1, "can": 1, "beside": 0, "the": 0, "pringles": 2, "on": 0, "the": 0, "counter": 1}


# interpretation_2
# "Take the coke can (which is somewhere else) and place it beside the pringles (that are already on the counter).",
intent_labels_2 = {"place": 0,"the": 0,"coke": 1,"can": 1,"beside": 0,"the": 0,"pringles": 1,"on": 0,"the": 0,"counter": 2}


In [4]:
instruction = input("Enter instruction: ")

In [5]:
passedin_intent_dict = {'Goal_intent': ['counter'],
 'Avoidance_intent': [],
 'Detail_intent': ['coke', 'can']}

In [6]:
baseline_choice = "intent_label"
passedin_intent_dict = build_intent_dict(intent_labels_1)
# baseline_choice = "asr"

baseline_prompt_path = f"prompts/{baseline_choice}_prompt.txt"


limp_llmplaner_incontext_prompt = open(baseline_prompt_path, "r").read() 
limp_planner_output = limp_llm_plan(limp_llmplaner_incontext_prompt,instruction, passedin_intent_dict, baseline=baseline_choice, verbose=False)

print("**************************************************\nInstruction:",instruction)
print(f"--------------------------------------------------\nLLM Planner Output\n--------------------------------------------------\n{limp_planner_output}\n**************************************************")


Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
**************************************************
Instruction: Pick up the book on the table with the red cover.
--------------------------------------------------
LLM Planner Output
--------------------------------------------------
Explanation: The goal is to pick up the coke can and place it on the counter. The coke can is next to the pringles.
Resolved referents with spatial predicates: ["coke::isnextto(pringles)"]
In my plan I will always stick to "coke::isnextto(pringles)" whenever i need to reference coke
Task plan:
1. goto[coke::isnextto(pringles)] 
2. pick[coke::isnextto(pringles)] 
3. goto[counter] 
4. place[coke::isnextto(pringles),counter]
**************************************************


In [8]:
json_output = generate_json_with_gpt(limp_planner_output)

print("**************************************************\nJson Output:")

# print json in tree format
import json
print(json.dumps(json_output, indent=4, sort_keys=True))

		Model: gpt-4o || Deterministic?: True
**************************************************
Json Output:


'"{\\"Interpretation\\":\\"The task is to move a coke can from its current location next to a Pringles can to a counter.\\",\\"Explanation\\":\\"The task involves identifying the location of the coke can, which is specified as being next to a Pringles can. The task plan involves navigating to the location of the coke can, picking it up, moving to the counter, and placing the coke can on the counter. The spatial relationship \'is next to\' is used to consistently identify the coke can\'s location relative to the Pringles can.\\",\\"ResolvedReferents\\":[\\"coke::isnextto(pringles)\\"],\\"TaskPlan\\":[\\"goto[coke::isnextto(pringles)]\\",\\"pick[coke::isnextto(pringles)]\\",\\"goto[counter]\\",\\"place[coke::isnextto(pringles),counter]\\"]}"'

## Util Functions

In [9]:
# define function to read json data
# after for each entry audio_file, sentence and intent dict
def build_task_plans(model_output_file, task_plan_file):
    # indicate you have started the function
    print("Building task plans...")
    # variable to store the task plan for each audio file
    task_plans = {}

    # define experiment baseline
    baseline_choice = "intent_label"
    baseline_prompt_path = f"prompts/{baseline_choice}_prompt.txt"

    # read the prompt
    limp_llmplaner_incontext_prompt = open(baseline_prompt_path, "r").read()

    # read the json file
    with open(model_output_file, encoding='utf-8') as f:
        data = json.load(f)

    # iterate over each entry in the json file
    for entry in data:
        # get the audio file name
        audio_file = entry['audio_file']

        # get the sentence
        instruction = entry['sentence']

        # get the intent dict
        intent_dict = entry['intent_dict']

        # generate the task plan
        limp_planner_output = limp_llm_plan(limp_llmplaner_incontext_prompt,instruction, intent_dict, baseline=baseline_choice, verbose=False)

        # generate json version of the task plan
        task_plan_json = generate_json_with_gpt(limp_planner_output)

        # store the task plan in the task_plans dictionary

        # add the audio file name as the key, and the instrction, task plan json as the value
        task_plans[audio_file] = {'instruction': instruction, 'task_plan': task_plan_json}
    
    # write the task plans to a file
    with open(task_plan_file, 'w') as f:
        json.dump(task_plans, f, indent=2, sort_keys=True)

    # indicate you have finished the function
    print("Task plans built and saved to file.")

       


In [11]:
# running for Bilstm model
model_output_file = "../ltl/data/filtered_prosody_bilstm_multiclass_results.json"
task_plan_file = "../ltl/data/bilstm_generated_task_plans.json"

build_task_plans(model_output_file, task_plan_file)

Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic